In [1]:
import struct
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras import backend as K

In [2]:
print(K.image_data_format())

channels_last


In [3]:
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

In [4]:
X_train = read_idx('/content/train-images.idx3-ubyte')
y_train = read_idx('/content/train-labels.idx1-ubyte')
X_test = read_idx('/content/t10k-images.idx3-ubyte')
y_test = read_idx('/content/t10k-labels.idx1-ubyte')

<ipython-input-3-6a24037f0b58>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)


In [5]:
X_train = X_train.reshape(X_train.shape[0], 28,28,1).astype('float32')
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')

In [6]:
X_train = X_train/255
X_test = X_test/255

In [7]:
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [8]:
num_pixels = X_train.shape[1] * X_train.shape[2]

In [10]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Flatten())
model.add(Dense(units=128, kernel_initializer ='uniform', activation='relu')) # Changed output_dim to units
model.add(Dense(units=10, kernel_initializer ='uniform', activation='softmax')) # Changed output_dim to units

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
early_stopping_monitor = EarlyStopping(patience=3)

In [13]:
model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=10, verbose=True, callbacks=[early_stopping_monitor])
scores = model.evaluate(X_test,y_test,verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7685 - loss: 0.7350 - val_accuracy: 0.9413 - val_loss: 0.1878
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9412 - loss: 0.1880 - val_accuracy: 0.9554 - val_loss: 0.1515
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9581 - loss: 0.1332 - val_accuracy: 0.9651 - val_loss: 0.1114
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9660 - loss: 0.1069 - val_accuracy: 0.9675 - val_loss: 0.1046
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9728 - loss: 0.0856 - val_accuracy: 0.9703 - val_loss: 0.0916
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9760 - loss: 0.0757 - val_accuracy: 0.9723 - val_loss: 0.0856
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9810 - loss: 0.0611 - val_accuracy: 0.9730 - val_loss: 0.0844
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9825 - loss: 0.0532 